In [2]:
import pandas as pd
import numpy as np
from epiweeks import Week
from numpy import log, exp
from scipy.stats import lognorm
from scipy.optimize import minimize
from matplotlib import use, get_backend
import aux_functions

#use('TkAgg', force=True)
from matplotlib import pyplot as plt
#print("Switched to:", get_backend())

Load preds: 

In [3]:
df_preds= pd.read_csv('data/preds_2nd_sprint.csv.gz')

df_preds = df_preds.loc[df_preds.model_id != 141]
df_preds.date = pd.to_datetime(df_preds.date)

df_preds = df_preds.loc[df_preds.date <= Week(2025, 40).startdate().strftime('%Y-%m-%d')]
            
df_preds.head()

/var/folders/ch/kxpr39wx44v97968yr_4hmch0000gn/T/ipykernel_30307/3044633114.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_preds= pd.read_csv('data/preds_2nd_sprint.csv.gz')


,date,lower_95,lower_90,lower_80,lower_50,pred,upper_50,upper_80,upper_90,upper_95,valid_test,state,model_id
0,2024-10-06,22.422222,40.304679,79.162970,244.274988,853.700012,2982.915845,9196.810733,18041.732884,32366.797486,3,RS,108
1,2024-10-13,18.743651,34.187301,68.269130,216.515891,779.997873,2809.263104,8900.886361,17748.532118,32294.740637,3,RS,108
2,2024-10-20,6.581569,12.226861,24.874887,81.165077,301.351891,1118.132411,3638.484729,7371.869282,13600.483733,3,RS,108
3,2024-10-27,17.097895,32.060474,66.082749,220.934380,843.905436,3222.679992,10763.205908,22149.610067,41420.387215,3,RS,108
4,2024-11-03,18.724941,35.572182,74.438413,255.270334,1003.039414,3940.407380,13500.169745,28209.239339,53454.977878,3,RS,108


### Estimate from the KL: 

In [3]:
l_pos = tuple([col for col in df_preds.columns if 'lower' in col])
u_pos = tuple([col for col in df_preds.columns if 'upper' in col])
the_pos = list(l_pos + u_pos)
l_pos = list(l_pos)
u_pos = list(u_pos)

lvls = (0.5, 0.8, 0.9, 0.95)
p = [aux_functions.quantile_pair(lvl) for lvl in lvls]
p = [i for quantile in p for i in quantile]
p.append(0.5)
ps = p.copy()
ps.sort()


In [4]:
ps

[0.025000000000000022,
 0.04999999999999999,
 0.09999999999999998,
 0.25,
 0.5,
 0.75,
 0.9,
 0.95,
 0.975]

In [5]:
k = 0
lowers = list(df_preds.loc[k, l_pos])
J = len(lowers)
names_l = [name.split('_') for name in df_preds.loc[k, l_pos].index]
extracted_lvls = [float(parts[1]) for parts in names_l if len(parts) > 1]

uppers = list(df_preds.loc[k, u_pos])

intervals = pd.DataFrame({
    'level': [i / 100 for i in extracted_lvls],
    'lwr': lowers,
    'med': [df_preds.loc[k, 'pred']] * J,
    'upr': uppers
})

approxs = [[], []]
for j in range(0, J):
    aux = aux_functions.get_lognormal_pars2(med = intervals['med'][j],
                                           lwr = intervals['lwr'][j],
                                           upr = intervals['upr'][j],
                                           conf_level = intervals['level'][j],
                                           fn_loss = 'lower')
    approxs[0].append(aux[0])
    approxs[1].append(aux[1])

ln_approx = pd.DataFrame({
    'level': list(intervals['level']),
    'mu': approxs[0],
    'sigma': approxs[1]
})

ln_approx.head()

,level,mu,sigma
0,0.95,5.555355,1.247626
1,0.90,6.411540,1.650647
2,0.80,7.085976,2.118110
3,0.50,7.941591,3.622437


aux = aux_functions.get_lognormal_pars(med = intervals['med'][j],
                                           lwr = intervals['lwr'][j],
                                           upr = intervals['upr'][j],
                                           alpha = intervals['level'][j])

aux

aux = aux_functions.get_lognormal_pars2(med = intervals['med'][j],
                                           lwr = intervals['lwr'][j],
                                           upr = intervals['upr'][j],
                                           conf_level = intervals['level'][j], fn_loss = 'median')

aux

print(intervals['lwr'][j], intervals['med'][j], intervals['upr'][j])

print(lognorm.ppf([0.25, 0.5, 0.75], scale = exp(7.9415914959428235),  s = 3.6224374131147576))
print(lognorm.ppf([0.25, 0.5, 0.75], scale = exp(6.74957983380822),  s = 5.389716825671165))


In [6]:
Opt_direct_KL = aux_functions.minimize_opt_fn1(method = 'L-BFGS-B',
                                               x0 = np.array([0.0, 0.0]),
                                               J = J,
                                               ln_approx = ln_approx)

direct_KL_pars = [Opt_direct_KL.x[0], exp(Opt_direct_KL.x[1])]


direct_KL_pars_closedForm = [sum(ln_approx['mu']/ln_approx['sigma']**2)/sum(1/ln_approx['sigma']**2), np.sqrt(J/sum(1/ln_approx['sigma']**2))]


In [7]:
direct_KL_pars

[6.195184475970171, 1.7483669487581006]

In [8]:
direct_KL_pars_closedForm

[6.19518424800456, 1.748366922182896]

In [9]:
ln_approx

,level,mu,sigma
0,0.95,5.555355,1.247626
1,0.90,6.411540,1.650647
2,0.80,7.085976,2.118110
3,0.50,7.941591,3.622437


In [10]:
df_preds.loc[0]

date          2024-10-06 00:00:00
lower_95                22.422222
lower_90                40.304679
lower_80                 79.16297
lower_50               244.274988
pred                   853.700012
upper_50              2982.915845
upper_80              9196.810733
upper_90             18041.732884
upper_95             32366.797486
valid_test                      3
state                          RS
model_id                      108
Name: 0, dtype: object

In [11]:
lognorm.ppf(ps, scale = exp(direct_KL_pars_closedForm[0]),  s = direct_KL_pars_closedForm[1])

array([   15.9338862 ,    27.64311177,    52.17283107,   150.796113  ,
         490.38178854,  1594.69825675,  4609.18630671,  8699.2484965 ,
       15092.00552609])

Condições fixas para todo o método: 

out['est'] = lognorm.ppf(ps,
                              scale = exp(cdf_fit[0]),  # scale = exp(meanlog)
                              s = cdf_fit[1])

out.head(10)

In [15]:
def process_row(row):
    # pegar valores da linha
    lowers = list(row[l_pos])
    J = len(lowers)
    names_l = [name.split('_') for name in row[l_pos].index]
    extracted_lvls = [float(parts[1]) for parts in names_l if len(parts) > 1]
    
    uppers = list(row[u_pos])
    
    intervals = pd.DataFrame({
        'level': [i / 100 for i in extracted_lvls],
        'lwr': lowers,
        'med': [row['pred']] * J,
        'upr': uppers
    })
    
    approxs = [[], []]
    for j in range(0, J):
        aux = aux_functions.get_lognormal_pars2(med = intervals['med'][j],
                                               lwr = intervals['lwr'][j],
                                               upr = intervals['upr'][j],
                                               conf_level = intervals['level'][j],
                                               fn_loss = 'lower')
        approxs[0].append(aux[0])
        approxs[1].append(aux[1])
    
    ln_approx = pd.DataFrame({
        'level': list(intervals['level']),
        'mu': approxs[0],
        'sigma': approxs[1]
    })

    #Opt_direct_KL = aux_functions.minimize_opt_fn1(method = 'L-BFGS-B',
    #                                           x0 = np.array([0.0, 0.0]),
    #                                           J = J,
    #                                           ln_approx = ln_approx)

    #direct_KL_pars = [Opt_direct_KL.x[0], exp(Opt_direct_KL.x[1])]

    direct_KL_pars_closedForm = [sum(ln_approx['mu']/ln_approx['sigma']**2)/sum(1/ln_approx['sigma']**2), np.sqrt(J/sum(1/ln_approx['sigma']**2))]
    
    return pd.Series({
    'mu': direct_KL_pars_closedForm[0],
    'sigma': direct_KL_pars_closedForm[1]
})

In [18]:
df_t = df_preds.head(5)

df_t.head()

,date,lower_95,lower_90,lower_80,lower_50,pred,upper_50,upper_80,upper_90,upper_95,valid_test,state,model_id
0,2024-10-06,22.422222,40.304679,79.162970,244.274988,853.700012,2982.915845,9196.810733,18041.732884,32366.797486,3,RS,108
1,2024-10-13,18.743651,34.187301,68.269130,216.515891,779.997873,2809.263104,8900.886361,17748.532118,32294.740637,3,RS,108
2,2024-10-20,6.581569,12.226861,24.874887,81.165077,301.351891,1118.132411,3638.484729,7371.869282,13600.483733,3,RS,108
3,2024-10-27,17.097895,32.060474,66.082749,220.934380,843.905436,3222.679992,10763.205908,22149.610067,41420.387215,3,RS,108
4,2024-11-03,18.724941,35.572182,74.438413,255.270334,1003.039414,3940.407380,13500.169745,28209.239339,53454.977878,3,RS,108


In [19]:
df_t[['mu', 'sigma']] = df_t.apply(process_row, axis=1)

df_t.head()

/var/folders/ch/kxpr39wx44v97968yr_4hmch0000gn/T/ipykernel_45479/3155731185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t[['mu', 'sigma']] = df_t.apply(process_row, axis=1)
/var/folders/ch/kxpr39wx44v97968yr_4hmch0000gn/T/ipykernel_45479/3155731185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t[['mu', 'sigma']] = df_t.apply(process_row, axis=1)


,date,lower_95,lower_90,lower_80,lower_50,pred,upper_50,upper_80,upper_90,upper_95,valid_test,state,model_id,mu,sigma
0,2024-10-06,22.422222,40.304679,79.162970,244.274988,853.700012,2982.915845,9196.810733,18041.732884,32366.797486,3,RS,108,6.195184,1.748367
1,2024-10-13,18.743651,34.187301,68.269130,216.515891,779.997873,2809.263104,8900.886361,17748.532118,32294.740637,3,RS,108,6.091297,1.790893
2,2024-10-20,6.581569,12.226861,24.874887,81.165077,301.351891,1118.132411,3638.484729,7371.869282,13600.483733,3,RS,108,5.124968,1.834564
3,2024-10-27,17.097895,32.060474,66.082749,220.934380,843.905436,3222.679992,10763.205908,22149.610067,41420.387215,3,RS,108,6.144043,1.872778
4,2024-11-03,18.724941,35.572182,74.438413,255.270334,1003.039414,3940.407380,13500.169745,28209.239339,53454.977878,3,RS,108,6.304380,1.912227


In [20]:
%%time 

df_preds[['mu', 'sigma']] = df_preds.apply(process_row, axis=1)

df_preds.head()

/Users/eduardoaraujo/Documents/validation_preds_sprint/harmonize_preds/aux_functions.py:74: RuntimeWarning: divide by zero encountered in scalar divide
  attained_loss = abs(upr - tent_qs[1]) / upr
/opt/homebrew/Caskroom/miniforge/base/envs/lstm_infodengue_sprint/lib/python3.12/site-packages/scipy/optimize/_optimize.py:869: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):
/Users/eduardoaraujo/Documents/validation_preds_sprint/harmonize_preds/aux_functions.py:74: RuntimeWarning: divide by zero encountered in scalar divide
  attained_loss = abs(upr - tent_qs[1]) / upr
/opt/homebrew/Caskroom/miniforge/base/envs/lstm_infodengue_sprint/lib/python3.12/site-packages/scipy/optimize/_optimize.py:869: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):
/Users/eduardoaraujo/Documents/validation_preds_sprint/harmonize_preds/aux_functions.py:74: RuntimeWarning: divide by zero encountered in scalar

CPU times: user 47min 2s, sys: 21.7 s, total: 47min 24s
Wall time: 48min 21s


,date,lower_95,lower_90,lower_80,lower_50,pred,upper_50,upper_80,upper_90,upper_95,valid_test,state,model_id,mu,sigma
0,2024-10-06,22.422222,40.304679,79.162970,244.274988,853.700012,2982.915845,9196.810733,18041.732884,32366.797486,3,RS,108,6.195184,1.748367
1,2024-10-13,18.743651,34.187301,68.269130,216.515891,779.997873,2809.263104,8900.886361,17748.532118,32294.740637,3,RS,108,6.091297,1.790893
2,2024-10-20,6.581569,12.226861,24.874887,81.165077,301.351891,1118.132411,3638.484729,7371.869282,13600.483733,3,RS,108,5.124968,1.834564
3,2024-10-27,17.097895,32.060474,66.082749,220.934380,843.905436,3222.679992,10763.205908,22149.610067,41420.387215,3,RS,108,6.144043,1.872778
4,2024-11-03,18.724941,35.572182,74.438413,255.270334,1003.039414,3940.407380,13500.169745,28209.239339,53454.977878,3,RS,108,6.304380,1.912227


In [21]:
df_preds.isnull().sum()

date          0
lower_95      0
lower_90      0
lower_80      0
lower_50      0
pred          0
upper_50      0
upper_80      0
upper_90      0
upper_95      0
valid_test    0
state         0
model_id      0
mu            0
sigma         0
dtype: int64

In [23]:
col = 'sigma'
pos_inf_count = np.isposinf(df_preds[col]).sum()

# count only negative inf
neg_inf_count = np.isneginf(df_preds[col]).sum()

# count both +inf and -inf
total_inf_count = np.isinf(df_preds[col]).sum()

print("positive inf:", pos_inf_count)
print("negative inf:", neg_inf_count)
print("total inf:", total_inf_count)

positive inf: 0
negative inf: 0
total inf: 0


In [24]:
df_preds.to_csv('results/preds_KL.csv', index = False)